In [1]:
%load_ext autoreload
%autoreload 2

IMPORTS
==================



In [2]:
import pandas as pd
import os
import pickle
from pyprojroot import here

In [3]:
from src.tauso.off_target.Roni.off_target_pipeline.off_target_functions import parse_gtf, parse_fasta

from src.tauso.off_target.Roni.off_target_pipeline.get_premRNA_sequences import final_func_premrna, enrich_expression_data_with_sequence

from src.tauso.off_target.Roni.off_target_pipeline.mutate_cell_line_transcriptome import get_expression_of_cell_line, get_mutations_of_cell_line, mutate, mutation_dict, celline_list, find_shift, mutate_transcriptome

DATA
====

Paths

In [4]:
PROJECT_ROOT = here()
DATA_DIR = os.path.join(PROJECT_ROOT, "src", "tauso", "off_target", "Roni", "data")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "src", "tauso", "off_target", "Roni", "outputs")

fasta_path = os.path.join(DATA_DIR, "GRCh38.p13.genome.fa")
gtf_path = os.path.join(DATA_DIR, "gencode.v34.chr_patch_hapl_scaff.annotation.gtf")
exp_path = os.path.join(DATA_DIR, "OmicsExpressionTPMLogp1HumanProteinCodingGenes.csv")
mut_path = os.path.join(DATA_DIR, "OmicsSomaticMutations.csv")

Select the cell-line(s) you work on

In [5]:
save_csv = True

In [6]:
cell_line_lst = ["ACH-000681"]
transcriptomes = {}

I. GET SEQUENCES
=

In [7]:
# Step 1 - Parse FASTA
chrom2seq = parse_fasta(fasta_path, "../outputs/fasta_pickle_output_file_path.pkl")

# Step 2 - Parse GTF
annotations = parse_gtf(gtf_path, "../outputs/gtf_pickle_output_file_path.pkl")

✅ Saved 639 main chromosomes to ../outputs/fasta_pickle_output_file_path.pkl
Read GTF file


In [8]:
# Step 3 - Expression and mutation data per cell line
for cell_line in cell_line_lst:
    exp_data = get_expression_of_cell_line(cell_line, exp_path, OUTPUT_DIR)
    mut_data = get_mutations_of_cell_line(cell_line, mut_path, OUTPUT_DIR)
    transcriptomes[cell_line] = [exp_data, mut_data]
    transcriptomes = enrich_expression_data_with_sequence(transcriptomes, cell_line, chrom2seq, annotations)
    mut_exp_data = mutate_transcriptome(exp_data, mut_data, annotations)
    transcriptomes[cell_line].append(mut_exp_data)
    if save_csv:
        for cell_line, data in transcriptomes.items():
            df = pd.DataFrame(data[-1])
            df.to_csv(os.path.join(OUTPUT_DIR, cell_line + "_transcriptome.csv"), index=False)


Successfully processed ACH-000681. Found 16047 expressed genes.
Obtained mutation data for ACH-000681
Building Gene -> Transcript index...
Processing ACH-000681...
Enrichment complete.
Mutating transcriptomes...
ENST00000378344.7 shift: 405
Skipping ENST00000378344.7 due to error: list assignment index out of range
ENST00000377205.6 shift: 38387
Skipping ENST00000377205.6 due to error: list assignment index out of range
ENST00000528266.6 shift: 1553
Skipping ENST00000528266.6 due to error: list assignment index out of range
ENST00000606738.3 shift: 195100
Skipping ENST00000606738.3 due to error: list assignment index out of range
ENST00000371208.5 shift: 54146
Skipping ENST00000371208.5 due to error: list assignment index out of range
ENST00000498273.2 shift: 13926
Skipping ENST00000498273.2 due to error: list assignment index out of range
ENST00000635253.2 shift: 170661
Skipping ENST00000635253.2 due to error: list assignment index out of range
ENST00000369541.4 shift: 24
Skipping ENS

In [9]:
transcriptomes["ACH-000681"][-1].head()

,Gene,ACH-000681_expression_norm,expression_TPM,Original Transcript Sequence,Mutated Transcript Sequence
Transcript_ID,,,,,
ENST00000362079.2,MT-CO3 (4514),15.250876,38990.6,ATGACCCACCAATCACATGCCTATCATATAGTAAAACCCAGCCCAT...,
ENST00000361624.2,MT-CO1 (4512),14.970982,32114.5,ATGTTCGCCGACCGTTGACTATTCTCTACAAACCACAAAGACATTG...,
ENST00000361739.1,MT-CO2 (4513),14.783070,28192.4,ATGGCACATGCAGCGCAAGTAGGTCTACAAGACGCTACTTCCCCTA...,
ENST00000361899.2,MT-ATP6 (4508),14.529565,23649.2,ATGAACGAAAATCTGTTCGCTTCATTCATTGCCCCCACAATCCTAG...,
ENST00000361381.2,MT-ND4 (4538),13.919915,15498.3,ATGCTAAAACTAATCGTCCCAACAATTATATTACTACCACTGACAT...,


II. MUTATE SEQUENCES
=

III. OFF-TARGET CALCULATION
=

In [11]:
exp_yes = transcriptomes["ACH-000681"][0]
exp_yes.to_csv("ACH-000681.transcriptome.csv", index=False)

In [13]:
hi = "AUGACCCACCAAUCACAUGCCUAUCAUAUAGUAAAACCCAGCCCAUGACCCCUAACAGGGGCCCUCUCAGCCCUCCUAAUGACCUCCGGCCUAGCCAUGUGAUUUCACUUCCACUCCAUAACGCUCCUCAUACUAGGCCUACUAACCAACACACUAACCAUAUACCAAUGAUGGCGCGAUGUAACACGAGAAAGCACAUACCAAGGCCACCACACACCACCUGUCCAAAAAGGCCUUCGAUACGGGAUAAUCCUAUUUAUUACCUCAGAAGUUUUUUUCUUCGCAGGAUUUUUCUGAGCCUUUUACCACUCCAGCCUAGCCCCUACCCCCCAAUUAGGAGGGCACUGGCCCCCAACAGGCAUCACCCCGCUAAAUCCCCUAGAAGUCCCACUCCUAAACACAUCCGUAUUACUCGCAUCAGGAGUAUCAAUCACCUGAGCUCACCAUAGUCUAAUAGAAAACAACCGAAACCAAAUAAUUCAAGCACUGCUUAUUACAAUUUUACUGGGUCUCUAUUUUACCCUCCUACAAGCCUCAGAGUACUUCGAGUCUCCCUUCACCAUUUCCGACGGCAUCUACGGCUCAACAUUUUUUGUAGCCACAGGCUUCCACGGACUUCACGUCAUUAUUGGCUCAACUUUCCUCACUAUCUGCUUCAUCCGCCAACUAAUAUUUCACUUUACAUCCAAACAUCACUUUGGCUUCGAAGCCGCCGCCUGAUACUGGCAUUUUGUAGAUGUGGUUUGACUAUUUCUGUAUGUCUCCAUCUAUUGAUGAGGGUCUU"

len(hi)

784

In [14]:
bye = "ATGACCCACCAATCACATGCCTATCATATAGTAAAACCCAGCCCATGACCCCTAACAGGGGCCCTCTCAGCCCTCCTAATGACCTCCGGCCTAGCCATGTGATTTCACTTCCACTCCATAACGCTCCTCATACTAGGCCTACTAACCAACACACTAACCATATACCAATGATGGCGCGATGTAACACGAGAAAGCACATACCAAGGCCACCACACACCACCTGTCCAAAAAGGCCTTCGATACGGGATAATCCTATTTATTACCTCAGAAGTTTTTTTCTTCGCAGGATTTTTCTGAGCCTTTTACCACTCCAGCCTAGCCCCTACCCCCCAATTAGGAGGGCACTGGCCCCCAACAGGCATCACCCCGCTAAATCCCCTAGAAGTCCCACTCCTAAACACATCCGTATTACTCGCATCAGGAGTATCAATCACCTGAGCTCACCATAGTCTAATAGAAAACAACCGAAACCAAATAATTCAAGCACTGCTTATTACAATTTTACTGGGTCTCTATTTTACCCTCCTACAAGCCTCAGAGTACTTCGAGTCTCCCTTCACCATTTCCGACGGCATCTACGGCTCAACATTTTTTGTAGCCACAGGCTTCCACGGACTTCACGTCATTATTGGCTCAACTTTCCTCACTATCTGCTTCATCCGCCAACTAATATTTCACTTTACATCCAAACATCACTTTGGCTTCGAAGCCGCCGCCTGATACTGGCATTTTGTAGATGTGGTTTGACTATTTCTGTATGTCTCCATCTATTGATGAGGGTCTT"

len(bye)

784